In [72]:
import os
import gc
import pyperclip as clip
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *
import selenium.webdriver as webdriver

% reload_ext autoreload
% autoreload 2

In [2]:
#PATH = os.path.abspath(r"S:\PUBLICO\Outorga 3º Andar\Provas de Radioamador\2018.07.25\Corrigidos")
PATH = os.path.abspath(r"C:\Users\rsilva\Desktop\SEI")
print(PATH)
SCAN = os.path.abspath(r"S:\PUBLICO\Outorga 3º Andar\Provas de Radioamador\2018.07.25\scaneados")

C:\Users\rsilva\Desktop\SEI


In [4]:
keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

In [3]:
def string_endereço(dados):
    
    d = {}
    
    s = 'A(o)<br><br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
    
    d["A"] = s
    
    return d

In [175]:
cpf = '391.315.248-26'
proc = r'53504.009825/2018-12'


# Scpx - Manipulação de Cadastro

In [183]:
driver = webdriver.Ie()
scpx = sistemas.Scpx(driver)

### Consulta

In [184]:
scpx.consulta(cpf)

KeyboardInterrupt: 

### Serviço

In [185]:
scpx.servico_incluir(cpf, num_processo=proc, silent=True)

In [ ]:
#scpx.servico_excluir(cpf, documento="2724547")
scpx.imprime_consulta(cpf)

## Estação

In [ ]:
scpx.prorrogar_rf(cpf)

In [ ]:
scpx.prorrogar_estacao(cpf)

In [186]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2A1983', sequencial='001')

In [153]:
scpx.incluir_estacao(cpf, 'Fixa', 'PX2A1972', sequencial='002')

TimeoutException: Message: 


In [187]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc)

In [7]:
scpx.movimento_transferir(cpf, 'A', 'G', proc=proc)

In [8]:
scpx.movimento_cancelar(cpf)

In [188]:
scpx.licenciar_estacao(cpf, silent=True)

In [189]:
scpx.licenciar_estacao(cpf, ppdess=False, silent=True)

In [190]:
dados = scpx.extrai_cadastro(cpf)

## Licença

In [ ]:
scpx.imprimir_licenca(cpf)

# SEI - Manipulação de Processo

## Autenticação

In [90]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [91]:
sei_ = sei.login_sei(browser, usr, pwd)

In [92]:
sei_ = sei.Sei(sei_)

In [23]:
sei_.itera_processos()

A página demorou muito tempo para carregar ou há somente 1 página de Processos


In [211]:
p = sei_.go_to_processo('53504.005262/2018-93')

## Doc Externo

In [208]:
files = sorted(os.listdir(PATH))
scaneados = sorted(os.listdir(SCAN))

In [209]:
file = os.path.join(PATH, files[-1])

In [210]:
p.incluir_doc_externo('E-mail', file, arvore="confirmando recebimento de Correspondência")

In [195]:
p.incluir_oficio('RC_OF_Sem_Contexto_PPDESS', string_endereço(dados))

In [ ]:
p.incluir_informe()

In [36]:
for prova, gabarito in zip(files, scaneados):

    nome = prova.split(".")[0]
    
    p.incluir_doc_externo("Prova", os.path.join(SCAN, gabarito), arvore=nome)
    
    sleep(5)
    

## Controles

In [201]:
p.edita_marcador(tipo="RC - Boleto Enviado", content='Até 13/10/2018')

In [112]:
p.edita_marcador()

In [197]:
p.edita_postit()

In [238]:
p.edita_postit("Analisado - Controlar pagamento e Emitir Licença", True)

In [35]:
p.excluir_acomp_especial()

In [75]:
p.concluir_processo()

## Testes

In [ ]:
tipo = "Outorga: Rádio do Cidadão"
atribuicao = 'rodrigoca'

In [ ]:
processos = [p for p in sei_.get_processos().values() if p['atribuicao'] == 'rodrigoca']

In [ ]:
mm = [p for p in processos if p['tipo'] == tipo]

In [ ]:
for p in mm:
    print(p.get('marcador'))

In [ ]:
concluir = list()

In [ ]:
for processo in processos:
    
    if 'Licença' in str(processo.get('marcador')):
        
        p = sei_.go_to_processo(processo['numero'])
        
        i = input()
        
        if i == '1': 
            
            concluir.append(processo)
            
            
        gc.collect()

In [ ]:
for processo in concluir:
    
    p = sei_.go_to_processo(processo['numero'])
    
    
    sleep(2)
    
    try:    
        p.edita_marcador()    
    
        p.edita_postit()
    
        p.excluir_acomp_especial()
        
        p.concluir_processo() 
            
    except:
            
            next

In [ ]:
#dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)


In [ ]:
for num, dado in tupla[30:]:
        
    p = sei_.go_to_processo(num)
    
    i = input()
    
    if i =='y':
        
        try:
            
            dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)
                               
            p.incluir_oficio('RC_OF_Sem_Contexto_Licença', string_endereço(dados))
            
            p.edita_marcador(tipo='RC - Aguardando AR/Resposta', content='Até 31/07/2018')        

            
        except:
            
            next            
            
    if i == 'c':
        
        p.edita_marcador()
        p.edita_postit()
        p.excluir_acomp_especial()
        p.concluir_processo() 

## Outros

In [ ]:
from bs4 import BeautifulSoup as soup
html = soup(scpx.driver.page_source, 'lxml')

In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for td in tr.find_all('td', string=True):
        key = td.text.strip(" :")
        value = td.find_next_sibling('td')            
        if value and key not in dict_ and value.text.strip() != "":
            dict_[key] = value.text.strip()
            print("{0} {1}".format(key, value.text.strip())) 


In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for label in tr.find_all('label', string=True):
        print("{0} {1}".format(label.attrs.get('id'), label.text.strip()))                           

In [ ]:
from bs4 import NavigableString
def next_is_string(tag):
    return isinstance(tag.next_element, NavigableString)             

# Boleto

In [ ]:
boleto = sistemas.Boleto(driver)

In [ ]:
fistel = '80113639805'
cpf = "31250732867"

In [ ]:
boleto.imprime_boleto(fistel, tipo_id='id_fistel')

In [239]:
gc.collect()

186006